In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import preprocessing

#read file
df = pd.read_csv("car_eval_train.csv")
df_test = pd.read_csv("car_eval_test.csv")

In [2]:
df_test.head()

,id,buying,maint,doors,persons,lug_boot,safety
0,0,low,vhigh,3,more,big,high
1,1,med,med,4,2,big,high
2,2,med,low,2,more,big,high
3,3,med,low,3,4,small,med
4,4,high,low,2,4,med,low


In [3]:
names = list(df.columns.values[:-1])
print(names)

#identifying columns   
for item in names:
    uniqueVals = df[item].unique()
    print(item+": ")
    print(uniqueVals)



['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
buying: 
['med' 'low' 'vhigh' 'high']
maint: 
['med' 'high' 'low' 'vhigh']
doors: 
['3' '5more' '2' '4']
persons: 
['4' '2' 'more']
lug_boot: 
['med' 'small' 'big']
safety: 
['med' 'high' 'low']


In [4]:
class_types = df.columns.values[-1]
df[class_types].value_counts()

class
unacc    968
acc      307
good      55
vgood     52
Name: count, dtype: int64

In [5]:
#encode into numbers for model

df_en = pd.get_dummies(df, columns=names, drop_first=True)
df_en = df_en.replace({True: 1, False: 0})
class_mapping = {"unacc": 0,
                 "acc": 1,
                 "good": 2,
                 "vgood": 3}
df_en['class'] = df_en['class'].map(class_mapping)

class_col = df_en['class']
df_en.drop(columns=['class'], inplace=True)
last_col_pos = df_en.columns.get_loc('safety_med') + 1
df_en.insert(last_col_pos, 'class', class_col)
df_en.head()

/var/folders/9p/37r_bxzx43lbmnmjw0dvtfx00000gn/T/ipykernel_98718/609437354.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_en = df_en.replace({True: 1, False: 0})


,buying_low,buying_med,buying_vhigh,maint_low,maint_med,maint_vhigh,doors_3,doors_4,doors_5more,persons_4,persons_more,lug_boot_med,lug_boot_small,safety_low,safety_med,class
0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,1
1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0
2,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1,1
3,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,2
4,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0


# Training the model
- Random Forest

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Read file
df = pd.read_csv("car_eval_train.csv")
df_test = pd.read_csv("car_eval_test.csv")

# Define column names (excluding the target class)
names = list(df.columns.values[:-1])

# Encode categorical features
df_en = pd.get_dummies(df, columns=names, drop_first=True)
df_en = df_en.replace({True: 1, False: 0})

# Encode target class
class_mapping = {"unacc": 0, "acc": 1, "good": 2, "vgood": 3}
df_en['class'] = df_en['class'].map(class_mapping)

# Move 'class' column to the last position
class_col = df_en.pop('class')
df_en['class'] = class_col

# Prepare training data
X = df_en.drop(columns=["class"])  # Features
y = df_en["class"]                 # Target

# Train Gradient Boosting model
best_gb_clf = GradientBoostingClassifier(
    learning_rate=0.2,
    max_depth=3,
    n_estimators=700,
    subsample=1.0,
    random_state=42
)
best_gb_clf.fit(X, y)

# Process test dataset
if 'df_test' in globals():
    test_ids = df_test["id"]  # Preserve test IDs
    X_test = pd.get_dummies(df_test.drop(columns=["id"]), columns=names, drop_first=True)
    
    # Ensure all features match the training set
    missing_cols = set(X.columns) - set(X_test.columns)
    for col in missing_cols:
        X_test[col] = 0  # Add missing columns with default value 0
    
    X_test = X_test[X.columns]  # Reorder columns to match training data

    # Predict on the test data
    test_preds = best_gb_clf.predict(X_test)

    # Convert numerical predictions back to original class labels
    reverse_class_mapping = {0: "unacc", 1: "acc", 2: "good", 3: "vgood"}
    test_preds = [reverse_class_mapping[pred] for pred in test_preds]

    # Create submission file with correct format
    submission = pd.DataFrame({"id": test_ids, "output": test_preds})
    submission.to_csv("submission.csv", index=False)
    print("Submission file saved as submission.csv")



KeyError: 'class'